# 1. Установка зависимостей

In [1]:
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install -q huggingface_hub llama-index transformers qdrant-client sentence-transformers vllm langchain llama-index-embeddings-huggingface llama_index-vector-stores-qdrant llama-index-llms-vllm langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

Для запуска нашей системы понадобится следующее
1. 3 модели - LLM, encoder и reranker. Все 3 модели подтягиваются из репозитория, откуда-либо их для демонстрации брать не нужно
2. База знаний для системы. Ее нужно взять из той же папки репозитория, где лежал этот нотубук - в данном случае это 30 инструкций

Для запуска на том сетапе моделей, что был нам выбран, вам нужно иметь 32 GB GPU (например, V100) \
В противном случае, можно поменять LLM на модель меньшего размера - однако в таком случае нет никакой гарантии качества ответов.

In [ ]:
import numpy as np
import os
import torch

from huggingface_hub import snapshot_download

from llama_index.core import Document
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import VectorStoreIndex
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.core.postprocessor import SimilarityPostprocessor
from llama_index.core import Settings
from transformers import AutoTokenizer
from qdrant_client import QdrantClient, models # for vector db, later
from sentence_transformers import CrossEncoder
from vllm import LLM, SamplingParams

from langchain_community.document_loaders import TextLoader, Docx2txtLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

from warnings import filterwarnings
filterwarnings('ignore')

In [ ]:
def file_to_chunks(file_name, sep, chunk_size, chunk_overlap):
    file_ext = file_name.split('.')[-1]
    file_path = file_name

    overall_chunks = []
    overall_pages = []

    # Загружаем содержимое файла
    if file_ext == 'txt':
        loader = TextLoader(file_path, encoding='utf-8')
    elif file_ext == 'docx':
        loader = Docx2txtLoader(file_path)
    elif file_ext == 'pdf':
        loader = PyPDFLoader(file_path)
    else:
        return
    file = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(
        separators = sep,
        chunk_size = chunk_size,
        chunk_overlap = chunk_overlap,
        length_function = len,
        is_separator_regex = False,
        add_start_index = False
    )

    # в начале каждой страницы есть таблица с данными о самом документе, компании и подобным вещам - мы удаляем их из каждого чанка, оставляя номер страницы
    for docs in file:
        content = docs.page_content
        chunks = text_splitter.split_text(content)

        page_number = chunks[0][chunks[0].find('Страница'):].strip().split('\n')[0].strip()
        overall_chunks.append(chunks[0][chunks[0].find('Версия') + 11:].strip())
        overall_pages.append(page_number)

    return overall_chunks, overall_pages

In [ ]:
def create_docs():
    documents = []
    sep = '\n'
    chunk_size = 2048
    chunk_overlap = 128

    for file in os.listdir('./docs_for_rag_v2'):
        file_name = os.path.join('./docs_for_rag_v2', file)
        try:
            chunks, pages = file_to_chunks(file_name, sep, chunk_size, chunk_overlap)
        except:
            print(file_name)

        for chunk, page in zip(chunks, pages):
            metadata = {
                "название документа": file,
                "страница в документе": page,
                "описание": chunk
            }

            documents.append(Document(text=chunk, metadata=metadata,
                             excluded_embed_metadata_keys=["название документа", "страница в документе"]))

    return documents


In [ ]:
docs = create_docs()

./docs_for_rag_v2/.ipynb_checkpoints


In [ ]:
len(docs)

1880

# 2. Загрузка моделей
В ячейке ниже будет производиться загрузка моделей для ассистента
Обратите внимание, что:
1. Без GPU загрузка будет невозможна
2. Процесс загрузки может занять достаточно продолжительное время (порядка 15 минут) - однако такой долгий запуск только при первой активации моделей

Также обратим внимание, что для запуска vLLM нужна особая версия модели saiga_llama3 - ее вы скачиваете при помощи строки snapshot_download

In [3]:
snapshot_download(repo_id="IlyaGusev/saiga_llama3_8b", revision="main_vllm", local_dir="./llm")

Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

.gitattributes:   0%|          | 0.00/1.85k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/689 [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/277 [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/446 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.9k [00:00<?, ?B/s]

'/content/llm'

К сожалению, после скачивания модели придется ручками вставить путь до модели в строчку ниже. Просим прощения за неудобства

Путь до модели вы можете найти в последней строке вывода предыдущей ячейки кода.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('./llm')
llm = LLM(
    model='./llm',
    dtype=torch.float16,
    gpu_memory_utilization=0.8,
    max_seq_len_to_capture=8192
)

Settings.embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-m3"
)

reranker = CrossEncoder('BAAI/bge-reranker-v2-m3')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


WARNING 06-16 03:33:13 config.py:1155] Casting torch.bfloat16 to torch.float16.
INFO 06-16 03:33:13 llm_engine.py:161] Initializing an LLM engine (v0.4.3) with config: model='./llm', speculative_config=None, tokenizer='./llm', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0, served_model_name=./llm)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 06-16 03:33:13 selector.py:120] Cannot use FlashAttention-2 backend for Volta and Turing GPUs.
INFO 06-16 03:33:13 selector.py:51] Using XFormers backend.
INFO 06-16 03:33:20 selector.py:120] Cannot use FlashAttention-2 backend for Volta and Turing GPUs.
INFO 06-16 03:33:20 selector.py:51] Using XFormers backend.
INFO 06-16 03:41:57 model_runner.py:146] Loading model weights took 14.9595 GB
INFO 06-16 03:42:02 gpu_executor.py:83] # GPU blocks: 4115, # CPU blocks: 2048
INFO 06-16 03:42:05 model_runner.py:854] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 06-16 03:42:05 model_runner.py:858] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 

In [ ]:
index = VectorStoreIndex.from_documents(documents=docs, show_progress=True)

Generating embeddings: 100%|██████████| 349/349 [00:12<00:00, 27.23it/s]


In [ ]:
retriever = index.as_retriever(similarity_top_k=7, node_postprocessors=[
                               SimilarityPostprocessor(similarity_cutoff=0.85)])

## Техническая особенность нашего решения - использование модели reranker

Основная задача реранкера - улучшить качество и релевантность результатов путем ранжирования чанков при помощи сравнения с запросом пользователя \
Проще говоря, реранкер ранжирует чанки из базы знаний, "выдвигая" наверх наиболее схожие с запросом пользователя чанки


Данный тип моделей работает сильно медленнее обычных encoder моделей, поэтому мы пошли на хитрость - мы подаем в реранкер не всю базу знаний, а только 7 изначально наиболее схожих фрагментов. Такой подход не замедляет работу решения, однако ощутимо помогает улучшить качество ответа.

In [ ]:
def top_k_rerank(query: str, retriever, reranker, top_k: int = 2):
    documents = retriever.retrieve(query)
    # relevant_score = max(doc.score for doc in documents)
    relevant_score = documents[0].score
    print(f'Наивысшее знаение релевантности документов: {relevant_score}')

    candidate_texts = [x.text for x in documents]
    candidate_names = [x.metadata['название документа'] for x in documents]
    candidate_pages = [x.metadata['страница в документе'] for x in documents]

    rerank_scores = reranker.predict(list(zip([query] * len(candidate_texts), candidate_texts)))
    ranked_indices = np.argsort(rerank_scores)[::-1]

    names = [candidate_names[i] for i in ranked_indices][:top_k]
    pages = [candidate_pages[i] for i in ranked_indices][:top_k]
    texts = [candidate_texts[i] for i in ranked_indices][:top_k]

    return names, pages, texts, relevant_score

## Вторая техническая особенность решения - использование специальных библиотек для инференса (предсказания) моделей

В нашем случае мы используем библиотеку vLLM - ее использование позволяет ускорить нам генерацию текста как минимум в 4 раза по сравнению с другими решениями

In [ ]:
def vllm_infer(
    tokenizer,
    wrapped_llm,
    texts,
    query,
    temperature: float = 0.2,
    top_p: float = 0.9,
    top_k: int = 30,
    max_tokens: int = 512,
    repetition_penalty: float = 1.1
):

    SYSTEM_PROMPT = "Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им."
    user_prompt = '''Используй только следующий контекст, чтобы кратко ответить на вопрос в конце.
        Не пытайся выдумывать ответ. Если контекст не соотносится с вопросом, скажи, что ты не можешь ответить на данный вопрос.
        Если вопрос не соотносится с банковской тематикой, выведи фразу "Я не могу ответить на ваш вопрос." и не выводи ничего больше.
        Контекст:
        ===========
        {texts}
        ===========
        Вопрос:
        ===========
        {query}'''.format(texts=texts, query=query)

    sampling_params = SamplingParams(
        temperature=temperature,
        top_p=top_p,
        top_k=top_k,
        max_tokens=max_tokens,
        repetition_penalty=repetition_penalty
    )

    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user_prompt}
    ]

    answers = []

    prompt = llm.llm_engine.tokenizer.tokenizer.apply_chat_template(conversation=messages, add_generation_prompt=True, tokenize=False)
    prompts = [prompt]

    outputs = llm.generate(prompts, sampling_params)


    for output in outputs:
        generated_text = output.outputs[0].text
        answers.append(generated_text)

    torch.cuda.empty_cache()
    return answers


## Дополнительные две (хоть и небольшие, но приятные хитрости) - "защита от дурака" и вывод дополнительной информации для пользователя

1. Порой люди могут кидать в систему запросы, которые никак не связаны с работой системы - на данные запросы крайне важно не давать "что попало" в качестве ответа, а передать информацию, что ассистент не может ответить на данный запрос.

2. Вывод дополнительной информации для пользователя сделан для того, чтобы пользователь смог сам из руководства, следуя картинкам и более подробно расписанным правилам, дополнить ответ бота именно теми знаниями, что нужны самому пользователю. Специально для этого мы выводим название наиболее релевантного документа и страницу для поиска информации.

In [ ]:
def response(query, retriever, reranker, tokenizer, llm) -> str:
    generated_text = '''
        {llm_gen}
        ===================================
        Источники дополнительной информации:
        Документ {doc_name}, {page_number}
        '''

    names, pages, chunks, relevant_score = top_k_rerank(query, retriever, reranker)

    if relevant_score >= 0.545:
        answer = vllm_infer(tokenizer, llm, query, chunks)

        if answer[0] == 'Я не могу ответить на ваш вопрос.':
            return answer[0]
        else:
            formatted_answer = generated_text.format(
                llm_gen=answer[0],
                doc_name=names[0], page_number=pages[0]
            )

            return formatted_answer


    else:
        return 'Данный вопрос выходит за рамки компетенций бота. Пожалуйста, переформулируйте вопрос или попросите вызвать сотрудника.'

## Примеры запросов к модели и ее ответы:

In [ ]:
query = 'Для чего предназначена вкладка «Начисления и взносы»?'
query_ans = response(query, retriever, reranker, tokenizer, llm)

print(query_ans)

Наивысшее знаение релевантности документов: 0.6646133211062943


Processed prompts: 100%|██████████| 1/1 [00:02<00:00,  2.03s/it, Generation Speed: 30.58 toks/s]


        Вкладка «Начисления и взносы» предназначена для отражения проводок по начисленной заработной плате, страховым взносам, резервам по отпускам, передаче резервов между филиалами.
        Источники дополнительной информации:
        Документ Инструкция_D_1C1_1_10_25_Учет_расчетов_по_заработной_плате3_НФ.pdf, Страница  12
        


In [ ]:
query = 'Почему звездные войны одна из самых популярных франшиз?'
query_ans = response(query, retriever, reranker, tokenizer, llm)

print(query_ans)

Наивысшее знаение релевантности документов: 0.27112051844419377
Данный вопрос выходит за рамки компетенций бота. Пожалуйста, переформулируйте вопрос или попросите вызвать сотрудника.


In [ ]:
query = 'какие формы печати есть для документа «Передача давальцу»?'
query_ans = response(query, retriever, reranker, tokenizer, llm)

print(query_ans)

Наивысшее знаение релевантности документов: 0.7253811161483722


Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  1.03it/s, Generation Speed: 32.02 toks/s]


        Документ «Передача давальцу» имеет две доступные печатные формы: ТОРГ-12 и М-15.
        Источники дополнительной информации:
        Документ Отражение_операций_по_давальческой_схеме_через_документ_«Заказ_давальца».pdf, Страница  22
        


In [ ]:
query = 'Какой порядок сопоставления документов рекомендуется при поступлении входящих электронных документов на агентское вознаграждение?'
query_ans = response(query, retriever, reranker, tokenizer, llm)

print(query_ans)

Наивысшее знаение релевантности документов: 0.7476941234235391


Processed prompts: 100%|██████████| 1/1 [00:08<00:00,  8.17s/it, Generation Speed: 36.60 toks/s]


        При поступлении входящих электронных документов на агентское вознаграждение рекомендуется следующий порядок сопоставления документов в обработке «Контур ЭДО»:

1. Если во входящем пакете с электронным УПД есть неформализованный документ вида «Отчет агента», необходимо выполнить сопоставление системного документа «Приобретение товаров и услуг» по агентскому вознаграждению как с входящим электронным УПД, так и с неформализованным документом вида «Отчет агента».

2. Аналогичное сопоставление необходимо выполнить, если неформализованный документ вида «Отчет агента» пришёл отдельным пакетом, не связанным с электронным УПД.

3. Если у входящего неформализованного документа вида «Отчет агента» и электронного УПД различаются подписанты или дата подписания, то электронный УПД сопоставляется с системным документом «Приобретение товаров и услуг» по агентскому вознаграждению, а неформализованный документ вида «Отчет агента» необходимо сопоставить с системным документом «Прочие ЭНД ЮЗ ЭДО»

In [ ]:
query = 'у меня в документе «Приобретение товаров и услуг» установлен признак «Дополнить сделку файлами после подписи». Че делать?'
query_ans = response(query, retriever, reranker, tokenizer, llm)

print(query_ans)

Наивысшее знаение релевантности документов: 0.8055366245704302


Processed prompts: 100%|██████████| 1/1 [00:02<00:00,  2.18s/it, Generation Speed: 35.39 toks/s]



        Для того, чтобы была возможность дополнения сделки файлами после подписания (утверждения) документа, в документе необходимо установить признак «Дополнить сделку файлами после подписи». Установить этот признак можно либо перед отправкой документа на согласование, либо на этапе согласования «ОЦО».
        Источники дополнительной информации:
        Документ Инструкция_D_1C1_1_19_01_Поступление_ТМЦ_на_склад_предприятия_от.pdf, Страница  92
        


In [ ]:
query = 'Сколько проблем в бизнесе?'
query_ans = response(query, retriever, reranker, tokenizer, llm)

print(query_ans)

Наивысшее знаение релевантности документов: 0.530224476217897
Данный вопрос выходит за рамки компетенций бота. Пожалуйста, переформулируйте вопрос или попросите вызвать сотрудника.
